In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [ ]:
cd /gdrive/My\ Drive/nn 

/gdrive/My Drive/nn


In [ ]:
%tensorflow_version 1.x
import argparse
import os
import pickle
import sys

import numpy as np

np.random.seed(7)  # for reproducibility

import tensorflow as tf
tf.random.set_random_seed(5005)

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils import class_weight


from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.layers import Dense, Flatten, Dropout
from tensorflow.python.keras.layers.convolutional import Conv1D
from tensorflow.python.keras.layers.pooling import MaxPooling1D
from tensorflow.python.keras.layers.pooling import AveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow.python.keras.backend as K
from tensorflow.keras.utils import to_categorical
from keras import regularizers

import matplotlib as mpl

mpl.use('Agg')
import utils
sys.path.append(".")

import importlib
importlib.reload(utils)

from utils import precision, recall_TP, load_data_merged, recall_TN, precision_N

l2_lam = 5e-07 
l1_lam = 1e-08 

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
def train_diff_model(data_path, res_path, model_name, input_len,
                     num_epoch, batchsize, func,model_path="./weights.hdf5", 
                     number_of_folds=1, save=True):
    print('creating model')
    model = func(input_len)
    print('compiling model')
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6)

    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy', recall_TP,recall_TN])
    checkpointer = ModelCheckpoint(filepath=model_path, verbose=1, save_best_only=True)
    earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

    print('loading data')
    x_train_list, y_train_list, x_valid_list, y_valid_list, x_test_seq, y_test = load_data_merged(data_path, input_len, kfold=number_of_folds)
    
    print("calculating weights")
    if number_of_folds != 1:
      print("the class weights won't work")

    print('fitting the model')
    for i in range(len(x_train_list)):
      print("Using fold %s/%s" %(i+1, number_of_folds))
      x_train_seq = x_train_list[i]
      y_train = y_train_list[i]
      x_valid_seq = x_valid_list[i]
      y_valid = y_valid_list[i]
      sample_weights = class_weight.compute_sample_weight('balanced', y_train)


      history = model.fit(x_train_seq, y_train, epochs=num_epoch, batch_size=batchsize,
                          validation_data=(x_valid_seq, y_valid), verbose=1,
                          callbacks=[checkpointer, earlystopper, ], sample_weight=sample_weights)  # tb])
    
    print("Finish training")
    if save:
      print('saving the model')
      model.save(os.path.join(res_path, model_name + ".h5"))

    print('testing the model')
    score = model.evaluate(x_test_seq, y_test)

    for i in range(len(model.metrics_names)):
        print(str(model.metrics_names[i]) + ": " + str(score[i]))

    return model

In [ ]:
model4 = train_diff_model(data_path=r"dataset/solo_test_with_crc10_var_0.05.pkl", 
                 res_path="./models", model_name="model_2nd_dense_dropout", model_path="./models/temp/1",
                 input_len=150, num_epoch=20, batchsize=128, number_of_folds=1,save=False, func = model26_another_chance_for_2nd_dense_more_dropout)

In [ ]:
model4.save(os.path.join("./models", "deepripe" + ".h5"))

In [ ]:
model4 = train_diff_model(data_path=r"dataset/solo_test_with_crc10_var_0.05.pkl", 
                 res_path="./models", model_name="model26_another_chance_for_2nd_dense_more_dropout_no_conv0", model_path="./models/temp/2",
                 input_len=150, num_epoch=20, batchsize=128, number_of_folds=1,save=False, func = model26_another_chance_for_2nd_dense_more_dropout_no_conv0)


creating model
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 150, 4)]          0         
_________________________________________________________________
conv1 (Conv1D)               (None, 148, 90)           1170      
_________________________________________________________________
pool1 (MaxPooling1D)         (None, 147, 90)           0         
_________________________________________________________________
drop1 (Dropout)              (None, 147, 90)           0         
_________________________________________________________________
conv2 (Conv1D)               (None, 143, 100)          45100     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 142, 100)          0         
_________________________________________________________________
dropout (Dropout)            (None, 142, 100) 

In [ ]:
model3 = train_diff_model(data_path=r"dataset/solo_test_with_crc10_var_0.05.pkl", 
                 res_path="./models", model_name="deepripe_model", model_path="./models/temp/3",
                 input_len=150, num_epoch=20, batchsize=128, number_of_folds=1,save=False, func = deepripe_orig)



creating model
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
left_input (InputLayer)      [(None, 150, 4)]          0         
_________________________________________________________________
left_conv1 (Conv1D)          (None, 144, 90)           2610      
_________________________________________________________________
left_pool1 (MaxPooling1D)    (None, 71, 90)            0         
_________________________________________________________________
left_drop1 (Dropout)         (None, 71, 90)            0         
_________________________________________________________________
conv_merged (Conv1D)         (None, 67, 100)           45100     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 12, 100)           0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 100)  

84468/84468 [==============================] - 8s 92us/sample - loss: 0.5735 - acc: 0.7038 - recall_TP: 0.6948 - recall_TN: 0.7374
loss: 0.5734771836795844
acc: 0.7038287
recall_TP: 0.6948228
recall_TN: 0.73737407

In [ ]:
def model26_another_chance_for_2nd_dense_and_avg(input_len):
    K.clear_session()
    # tf.random.set_random_seed(5005)

    input_node = Input(shape=(input_len, 4), name="input")
    conv0 = Conv1D(filters=90, kernel_size=1, padding='valid', activation="relu", name="conv0",kernel_regularizer=regularizers.l2(l2_lam))(input_node)
    conv1 = Conv1D(filters=90, kernel_size=3, padding='valid', activation="relu", name="conv1",kernel_regularizer=regularizers.l2(l2_lam))(conv0)
    pool1 = AveragePooling1D(pool_size=2, strides=1, name="pool1")(conv1)
    drop1 = Dropout(0.25, name="drop1")(pool1)
  
    conv2 = Conv1D(filters=100, kernel_size=5, padding='valid', activation="relu", name="conv2", kernel_regularizer=regularizers.l2(l2_lam))(drop1)
    pool2 = AveragePooling1D(pool_size=2, strides=1)(conv2)
    drop2 = Dropout(0.25)(pool2)
    flat = Flatten()(drop2)

    hidden1 = Dense(500, activation='relu', name="hidden1",kernel_regularizer=regularizers.l1(l1_lam))(flat)
    drop3 = Dropout(0.25)(hidden1)
    hidden2 = Dense(250, activation='relu', name="hidden2",kernel_regularizer=regularizers.l1(l1_lam))(drop3)

    output = Dense(1, activation='sigmoid', name="output")(hidden2)
    model = Model(inputs=[input_node], outputs=output)
    print(model.summary())
    return model

In [ ]:
def model26_another_chance_for_2nd_dense_more_dropout(input_len):
    K.clear_session()
    # tf.random.set_random_seed(5005)

    input_node = Input(shape=(input_len, 4), name="input")
    conv0 = Conv1D(filters=90, kernel_size=1, padding='valid', activation="relu", name="conv0",kernel_regularizer=regularizers.l2(l2_lam))(input_node)
    conv1 = Conv1D(filters=90, kernel_size=3, padding='valid', activation="relu", name="conv1",kernel_regularizer=regularizers.l2(l2_lam))(conv0)
    pool1 = MaxPooling1D(pool_size=2, strides=1, name="pool1")(conv1)
    drop1 = Dropout(0.25, name="drop1")(pool1)
  
    conv2 = Conv1D(filters=100, kernel_size=5, padding='valid', activation="relu", name="conv2", kernel_regularizer=regularizers.l2(l2_lam))(drop1)
    pool2 = MaxPooling1D(pool_size=2, strides=1)(conv2)
    drop2 = Dropout(0.25)(pool2)
    flat = Flatten()(drop2)

    hidden1 = Dense(500, activation='relu', name="hidden1",kernel_regularizer=regularizers.l1(l1_lam))(flat)
    drop3 = Dropout(0.5)(hidden1)
    hidden2 = Dense(250, activation='relu', name="hidden2",kernel_regularizer=regularizers.l1(l1_lam))(drop3)

    output = Dense(1, activation='sigmoid', name="output")(hidden2)
    model = Model(inputs=[input_node], outputs=output)
    print(model.summary())
    return model

In [ ]:
def model26_another_chance_for_2nd_dense_more_dropout_no_conv0(input_len):
    K.clear_session()
    # tf.random.set_random_seed(5005)

    input_node = Input(shape=(input_len, 4), name="input")
    conv1 = Conv1D(filters=90, kernel_size=3, padding='valid', activation="relu", name="conv1",kernel_regularizer=regularizers.l2(l2_lam))(input_node)
    pool1 = MaxPooling1D(pool_size=2, strides=1, name="pool1")(conv1)
    drop1 = Dropout(0.25, name="drop1")(pool1)
  
    conv2 = Conv1D(filters=100, kernel_size=5, padding='valid', activation="relu", name="conv2", kernel_regularizer=regularizers.l2(l2_lam))(drop1)
    pool2 = MaxPooling1D(pool_size=2, strides=1)(conv2)
    drop2 = Dropout(0.25)(pool2)
    flat = Flatten()(drop2)

    hidden1 = Dense(500, activation='relu', name="hidden1",kernel_regularizer=regularizers.l1(l1_lam))(flat)
    drop3 = Dropout(0.5)(hidden1)
    hidden2 = Dense(250, activation='relu', name="hidden2",kernel_regularizer=regularizers.l1(l1_lam))(drop3)

    output = Dense(1, activation='sigmoid', name="output")(hidden2)
    model = Model(inputs=[input_node], outputs=output)
    print(model.summary())
    return model

In [ ]:
def model26_another_chance_for_2nd_dense_more_filters(input_len):
    K.clear_session()
    # tf.random.set_random_seed(5005)

    input_node = Input(shape=(input_len, 4), name="input")
    conv0 = Conv1D(filters=150, kernel_size=1, padding='valid', activation="relu", name="conv0",kernel_regularizer=regularizers.l2(l2_lam))(input_node)
    conv1 = Conv1D(filters=150, kernel_size=3, padding='valid', activation="relu", name="conv1",kernel_regularizer=regularizers.l2(l2_lam))(conv0)
    pool1 = MaxPooling1D(pool_size=2, strides=1, name="pool1")(conv1)
    drop1 = Dropout(0.25, name="drop1")(pool1)
  
    conv2 = Conv1D(filters=300, kernel_size=5, padding='valid', activation="relu", name="conv2", kernel_regularizer=regularizers.l2(l2_lam))(drop1)
    pool2 = MaxPooling1D(pool_size=2, strides=1)(conv2)
    drop2 = Dropout(0.25)(pool2)
    flat = Flatten()(drop2)

    hidden1 = Dense(500, activation='relu', name="hidden1",kernel_regularizer=regularizers.l1(l1_lam))(flat)
    drop3 = Dropout(0.25)(hidden1)
    hidden2 = Dense(250, activation='relu', name="hidden2",kernel_regularizer=regularizers.l1(l1_lam))(drop3)

    output = Dense(1, activation='sigmoid', name="output")(hidden2)
    model = Model(inputs=[input_node], outputs=output)
    print(model.summary())
    return model

In [ ]:
def model26_another_chance_for_2nd_dense_more_dropout_and_avg(input_len):
    K.clear_session()
    # tf.random.set_random_seed(5005)

    input_node = Input(shape=(input_len, 4), name="input")
    conv0 = Conv1D(filters=90, kernel_size=1, padding='valid', activation="relu", name="conv0",kernel_regularizer=regularizers.l2(l2_lam))(input_node)
    conv1 = Conv1D(filters=90, kernel_size=3, padding='valid', activation="relu", name="conv1",kernel_regularizer=regularizers.l2(l2_lam))(conv0)
    pool1 = AveragePooling1D(pool_size=2, strides=1, name="pool1")(conv1)
    drop1 = Dropout(0.25, name="drop1")(pool1)
  
    conv2 = Conv1D(filters=100, kernel_size=5, padding='valid', activation="relu", name="conv2", kernel_regularizer=regularizers.l2(l2_lam))(drop1)
    pool2 = AveragePooling1D(pool_size=2, strides=1)(conv2)
    drop2 = Dropout(0.25)(pool2)
    flat = Flatten()(drop2)

    hidden1 = Dense(500, activation='relu', name="hidden1",kernel_regularizer=regularizers.l1(l1_lam))(flat)
    drop3 = Dropout(0.5)(hidden1)
    hidden2 = Dense(250, activation='relu', name="hidden2",kernel_regularizer=regularizers.l1(l1_lam))(drop3)

    output = Dense(1, activation='sigmoid', name="output")(hidden2)
    model = Model(inputs=[input_node], outputs=output)
    print(model.summary())
    return model

In [ ]:
def model26_avg_pool(input_len):
    K.clear_session()

    input_node = Input(shape=(input_len, 4), name="input")
    conv0 = Conv1D(filters=90, kernel_size=1, padding='valid', activation="relu", name="conv0",kernel_regularizer=regularizers.l2(l2_lam))(input_node)
    conv1 = Conv1D(filters=90, kernel_size=3, padding='valid', activation="relu", name="conv1",kernel_regularizer=regularizers.l2(l2_lam))(conv0)
    pool1 = AveragePooling1D(pool_size=2, strides=1, name="pool1")(conv1)
    drop1 = Dropout(0.25, name="drop1")(pool1)
  
    conv2 = Conv1D(filters=100, kernel_size=5, padding='valid', activation="relu", name="conv2",kernel_regularizer=regularizers.l2(l2_lam))(drop1)
    pool2 = AveragePooling1D(pool_size=2, strides=1)(conv2)
    drop2 = Dropout(0.25)(pool2)
    flat = Flatten()(drop2)

    hidden1 = Dense(500, activation='relu', name="hidden1",kernel_regularizer=regularizers.l1(l1_lam))(flat)
    output = Dense(1, activation='sigmoid', name="output")(hidden1)
    model = Model(inputs=[input_node], outputs=output)
    print(model.summary())
    return model

In [ ]:
def model26(input_len):
    K.clear_session()
    # tf.random.set_random_seed(5005)

    input_node = Input(shape=(input_len, 4), name="input")
    conv0 = Conv1D(filters=90, kernel_size=1, padding='valid', activation="relu", name="conv0",kernel_regularizer=regularizers.l2(l2_lam))(input_node)
    conv1 = Conv1D(filters=90, kernel_size=3, padding='valid', activation="relu", name="conv1",kernel_regularizer=regularizers.l2(l2_lam))(conv0)
    pool1 = MaxPooling1D(pool_size=2, strides=1, name="pool1")(conv1)
    drop1 = Dropout(0.25, name="drop1")(pool1)
  
    conv2 = Conv1D(filters=100, kernel_size=5, padding='valid', activation="relu", name="conv2",kernel_regularizer=regularizers.l2(l2_lam))(drop1)
    pool2 = MaxPooling1D(pool_size=2, strides=1)(conv2)
    drop2 = Dropout(0.25)(pool2)
    flat = Flatten()(drop2)

    hidden1 = Dense(500, activation='relu', name="hidden1",kernel_regularizer=regularizers.l1(l1_lam))(flat)
    output = Dense(1, activation='sigmoid', name="output")(hidden1)
    model = Model(inputs=[input_node], outputs=output)
    print(model.summary())
    return model

In [ ]:
def model26_another_chance_for_2nd_dense(input_len):
    K.clear_session()
    # tf.random.set_random_seed(5005)

    input_node = Input(shape=(input_len, 4), name="input")
    conv0 = Conv1D(filters=90, kernel_size=1, padding='valid', activation="relu", name="conv0",kernel_regularizer=regularizers.l2(l2_lam))(input_node)
    conv1 = Conv1D(filters=90, kernel_size=3, padding='valid', activation="relu", name="conv1",kernel_regularizer=regularizers.l2(l2_lam))(conv0)
    pool1 = MaxPooling1D(pool_size=2, strides=1, name="pool1")(conv1)
    drop1 = Dropout(0.25, name="drop1")(pool1)
  
    conv2 = Conv1D(filters=100, kernel_size=5, padding='valid', activation="relu", name="conv2", kernel_regularizer=regularizers.l2(l2_lam))(drop1)
    pool2 = MaxPooling1D(pool_size=2, strides=1)(conv2)
    drop2 = Dropout(0.25)(pool2)
    flat = Flatten()(drop2)

    hidden1 = Dense(500, activation='relu', name="hidden1",kernel_regularizer=regularizers.l1(l1_lam))(flat)
    drop3 = Dropout(0.25)(hidden1)
    hidden2 = Dense(250, activation='relu', name="hidden2",kernel_regularizer=regularizers.l1(l1_lam))(drop3)

    output = Dense(1, activation='sigmoid', name="output")(hidden2)
    model = Model(inputs=[input_node], outputs=output)
    print(model.summary())
    return model

In [ ]:
def model26_another_chance_for_2nd_dense_and_avg(input_len):
    K.clear_session()
    # tf.random.set_random_seed(5005)

    input_node = Input(shape=(input_len, 4), name="input")
    conv0 = Conv1D(filters=90, kernel_size=1, padding='valid', activation="relu", name="conv0",kernel_regularizer=regularizers.l2(l2_lam))(input_node)
    conv1 = Conv1D(filters=90, kernel_size=3, padding='valid', activation="relu", name="conv1",kernel_regularizer=regularizers.l2(l2_lam))(conv0)
    pool1 = AveragePooling1D(pool_size=2, strides=1, name="pool1")(conv1)
    drop1 = Dropout(0.25, name="drop1")(pool1)
  
    conv2 = Conv1D(filters=100, kernel_size=5, padding='valid', activation="relu", name="conv2", kernel_regularizer=regularizers.l2(l2_lam))(drop1)
    pool2 = AveragePooling1D(pool_size=2, strides=1)(conv2)
    drop2 = Dropout(0.25)(pool2)
    flat = Flatten()(drop2)

    hidden1 = Dense(500, activation='relu', name="hidden1",kernel_regularizer=regularizers.l1(l1_lam))(flat)
    drop3 = Dropout(0.25)(hidden1)
    hidden2 = Dense(250, activation='relu', name="hidden2",kernel_regularizer=regularizers.l1(l1_lam))(drop3)

    output = Dense(1, activation='sigmoid', name="output")(hidden2)
    model = Model(inputs=[input_node], outputs=output)
    print(model.summary())
    return model

In [ ]:
def deepripe_model(input_len):
    K.clear_session()
    # tf.random.set_random_seed(5005)

    input_node = Input(shape=(input_len, 4), name="input")
    conv1 = Conv1D(filters=90, kernel_size=7, padding='valid', activation="relu", name="conv1")(input_node)
    pool1 = MaxPooling1D(pool_size=4, strides=2, name="pool1")(conv1)
    drop1 = Dropout(0.25, name="drop1")(pool1)

    conv2 = Conv1D(filters=100, kernel_size=5, padding='valid', activation="relu", name="conv2")(drop1)
    pool2  = MaxPooling1D(pool_size=10, strides=5)(conv2)
    drop2  = Dropout(0.25)(pool2)
    flat = Flatten()(drop2)

    hidden1 = Dense(250, activation='relu', name="hidden1")(flat)
    output = Dense(1, activation='sigmoid', name="output")(hidden1)
    model = Model(inputs=[input_node], outputs=output)
    print(model.summary())
    return model



In [ ]:
def model_baseline(input_len):
    K.clear_session()
    # tf.random.set_random_seed(5005)

    input_node = Input(shape=(input_len, 4), name="input")
    conv1 = Conv1D(filters=90, kernel_size=7, padding='valid', activation="relu", name="conv1")(input_node)
    pool1 = MaxPooling1D(pool_size=4, strides=2, name="pool1")(conv1)
    drop1 = Dropout(0.25, name="drop1")(pool1)
  
    conv2 = Conv1D(filters=100, kernel_size=5, padding='valid', activation="relu", name="conv2")(drop1)
    pool2 = MaxPooling1D(pool_size=10, strides=5)(conv2)
    drop2 = Dropout(0.25)(pool2)
    flat = Flatten()(drop2)

    hidden1 = Dense(500, activation='relu', name="hidden1")(flat)
    output = Dense(1, activation='sigmoid', name="output")(hidden1)
    model = Model(inputs=[input_node], outputs=output)
    print(model.summary())
    return model

In [ ]:
def deepripe_orig(input_len_l):
  K.clear_session()
  left_dim=4
  right_dim=4
  num_units=50
  input_l=input_len_l

  nb_f_l=[90,100]
  f_len_l=[7,7]
  p_len_l=[4,10]
  s_l=[2,5]
  nb_f_r=[90,100]
  f_len_r=[7,7]
  p_len_r=[10,10]
  s_r=[5,5]

  left_input = Input(shape=(input_l,left_dim),name="left_input")

  left_conv1 = Conv1D(filters=nb_f_l[0],kernel_size=f_len_l[0], padding='valid',activation="relu",name="left_conv1")(left_input)
  left_pool1 = MaxPooling1D(pool_size=p_len_l[0], strides=s_l[0],name="left_pool1")(left_conv1)
  left_drop1 = Dropout(0.25,name="left_drop1")(left_pool1)

  conv_merged = Conv1D(filters=100,kernel_size= 5, padding='valid',activation="relu",name="conv_merged")(left_drop1)
  merged_pool = MaxPooling1D(pool_size=10, strides=5)(conv_merged)
  merged_drop = Dropout(0.25)(merged_pool)
  merged_flat = Flatten()(merged_drop)

  hidden1 = Dense(250, activation='relu',name="hidden1")(merged_flat)
  output = Dense(1, activation='sigmoid',name="output")(hidden1)
  model = Model(inputs=[left_input], outputs=output)
  print(model.summary())
  return model

Play
